In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import time

from pickle import load
import tensorflow as tf
from tensorflow import keras
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
physical_devices = tf.config.list_physical_devices('GPU')    
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

tf.random.set_seed(42)
K = keras.backend

from sklearn.preprocessing import StandardScaler,PowerTransformer
import math
from scipy.stats import multivariate_normal
from scipy.stats import norm
import scipy.stats as st

import os

# CI #

## Typical method

In [19]:
# No TL

loglik_array = np.load("no_tl_models/loglik_array.npy")[:,1] / 30000

In [20]:
interval = st.t.interval(confidence=0.95, df=len(loglik_array)-1, loc=np.mean(loglik_array), scale=st.sem(loglik_array))

np.mean(loglik_array) 

37.739235

In [14]:
interval

(21.234684287822464, 54.24378556081035)

In [15]:
interval[0] - np.mean(loglik_array)

-16.504550636493942

In [16]:
# No TL

loglik_array = np.load("tl_models/loglik_array.npy")[:,1] / 30000

interval = st.t.interval(confidence=0.95, df=len(loglik_array)-1, loc=np.mean(loglik_array), scale=st.sem(loglik_array))

np.mean(loglik_array) 

91.7293

In [17]:
interval

(91.10982801299332, 92.34877489228012)

In [18]:
interval[0] - np.mean(loglik_array)

-0.6194734396434001

## Alternative method

In [26]:
def rx_star(array):
    return np.random.choice(array,size=len(array))


In [27]:
# No TL

loglik_array = np.load("no_tl_models/loglik_array.npy")
loglik_array

array([[ 1.6753729e+03,  1.3989975e+06],
       [ 2.2109958e+03,  1.9560465e+06],
       [ 1.7163282e+03,  1.6983150e+06],
       [ 2.7829414e+03,  1.4063108e+06],
       [ 4.2518799e+03,  1.4718738e+06],
       [-1.1344034e+03, -9.0117445e+04],
       [ 5.3731134e+02, -2.9211159e+05],
       [ 5.8165449e+03,  2.1455115e+06],
       [-7.6814703e+02, -2.1399133e+05],
       [ 1.4390370e+03,  7.1822850e+05],
       [ 4.1379209e+03,  1.4822712e+06],
       [ 2.5738913e+02,  1.8782148e+06],
       [ 4.0522866e+03,  1.9413259e+06],
       [-1.2877104e+03, -1.9659972e+05],
       [ 4.4812061e+03,  1.6783796e+06]], dtype=float32)

In [28]:
a_sample = [np.mean(rx_star(loglik_array[:,1])) for _ in range(10000)]

In [29]:
np.mean(a_sample)

1132843.9

In [30]:
np.mean(a_sample) / 30000

37.7614625

In [7]:


lo, hi = np.quantile(a_sample,[0.025,0.975])
print(lo)
print(hi)



666083.1421875
1553599.2718749999


In [8]:
print(lo/30000)
print(hi/30000)

22.20277140625
51.786642395833326


In [9]:
# TL

loglik_array = np.load("tl_models/loglik_array.npy")
loglik_array

array([[   9447.353, 2779064.2  ],
       [   9129.026, 2678161.5  ],
       [   9420.567, 2713152.   ],
       [   9119.233, 2703558.   ],
       [   9398.688, 2734757.8  ],
       [   9459.448, 2763107.2  ],
       [   9522.136, 2778008.2  ],
       [   9453.144, 2765898.5  ],
       [   9365.953, 2756722.   ],
       [   9540.198, 2792608.2  ],
       [   9400.421, 2744609.5  ],
       [   9470.63 , 2775609.5  ],
       [   9500.283, 2794984.   ],
       [   9282.673, 2760949.5  ],
       [   9234.495, 2736996.   ]], dtype=float32)

In [10]:
a_sample = [np.mean(rx_star(loglik_array[:,1])) for _ in range(10000)]

In [11]:
np.mean(a_sample)

2751833.5

In [12]:
np.mean(a_sample) / 30000

91.72778333333333

In [13]:


lo, hi = np.quantile(a_sample,[0.025,0.975])
print(lo)
print(hi)



2734454.96875
2767328.0625


In [14]:
print(lo/30000)
print(hi/30000)

91.14849895833333
92.24426875


# Data loading

In [2]:
hr_data = np.load("ks_coarse_grained.npy")

data = np.mean(hr_data.reshape((hr_data.shape[0],20,5),order="C"),axis=-1)

training_data = data[:10000,:]

hr_data_2 = np.load("ks_coarse_grained_larger.npy")
data2 = np.mean(hr_data_2.reshape((hr_data_2.shape[0],20,5),order="C"),axis=-1)
weather = data2[50000:]

weather_scaled = (weather-training_data.mean())/training_data.std()
weather_scaled = np.expand_dims(weather_scaled,axis=1)

truth_set_full = weather_scaled[:,0].copy()
truth_list = [weather_scaled[:,0]]

# Model loading

In [3]:
hidden_state_size = 8


## No TL

In [4]:
index = 7

rnn_no_tl = keras.models.load_model("no_tl_models/{}.h5".format(index))

2022-09-11 17:07:08.485551: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-11 17:07:09.158076: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5699 MB memory:  -> device: 0, name: Quadro P4000, pci bus id: 0000:3b:00.0, compute capability: 6.1


In [5]:
sigma_rnn_no_tl =0.0019546913

## Tl

In [6]:
index = 9

rnn_tl = keras.models.load_model("tl_models/{}.h5".format(index))

In [7]:
sigma_rnn_tl = 0.001787844

# Weather analysis

In [8]:
num_init_conds = 500
length_simulations = 500

weather_array = np.zeros(shape=(num_init_conds,length_simulations,20))

a = 0
for i in truth_list:
    num_seq = int(i.shape[0]/truth_set_full.shape[0] * num_init_conds)
    for j in range(num_seq):
        starting_index = np.random.randint(0,i.shape[0]-length_simulations)
        seq = i[starting_index:starting_index+length_simulations,:]
        weather_array[a,:,:] = seq
        a = a+1

weather_array = weather_array[:a,:,:]

truth_weather = weather_array.transpose([0,2,1])

np.save("truth_weather.npy",truth_weather)

starting_condition = weather_array[:,0]

num_ensemble_members = 40
num_steps = length_simulations

In [9]:
def weather_simulation(starting_condition,num_steps,num_ensemble_members,rnn,sigma):
    
#     starting_condition = (starting_condition - training_data_mean)/training_data_scale
    initX = starting_condition.repeat(num_ensemble_members,axis=0)
    length = initX.shape[0]
    array = np.zeros(shape=(length,20,num_steps))
    array[:,:,0] = initX
    
    hidden_state1 = np.zeros(shape=(length,hidden_state_size))
    
    noise = np.random.normal(size=(length,20,num_steps))
    
    dummy_x = np.zeros(())
    dummy_hr = np.zeros(())
    
    for i in range(num_steps-1):
        print(i)
        x_mean,_,h1 = rnn([np.expand_dims(array[:,:,i],axis=1),dummy_x,dummy_hr,dummy_hr,hidden_state1])
        x_new = x_mean[:,0,:] + noise[:,:,i]*sigma
        array[:,:,i+1] = x_new
        hidden_state1 = h1
    

#     array= (array.transpose([0,2,1])*training_data_scale + training_data_mean).transpose([0,2,1])
    return   array.reshape(starting_condition.shape[0],num_ensemble_members,20,num_steps)
    

In [10]:
array_no_tl = weather_simulation(starting_condition,num_steps,num_ensemble_members,rnn_no_tl,sigma_rnn_no_tl)

0


2022-09-11 17:07:19.077905: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
np.save("weather_no_tl.npy",array_no_tl)

In [12]:
array_tl = weather_simulation(starting_condition,num_steps,num_ensemble_members,rnn_tl,sigma_rnn_tl)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
np.save("weather_tl.npy",array_tl)